<a href="https://colab.research.google.com/github/hyunicecream/Natural-Language-Processing-NLP-/blob/main/%EC%BD%94%EC%82%AC%EC%9D%B8%EC%9C%A0%EC%82%AC%EB%8F%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
문서1 = ['저는 사과 좋아요']
문서2 = ['저는 바나나 좋아요']
문서3 = ['저는 바나나 좋아요 저는 바나나 좋아요']

In [72]:
# 행렬의 곱셈을 사용하기 위해 dot사용
from numpy import dot
# 문서1, 문서2, 문서3에 들어있는 단어들의 각가의 거리를 계산하여 최단거리를 계산하기 위해 사용
from numpy.linalg import norm
# norm : 벡터 공간에서 벡터의  크기 또는 길이를 측정할 때 사용되는 개념
import numpy as np

# 코사인 유사도를 계산하는 함수를 만들기
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

# BoW를 만들어서 코사인 유사도를 계산한다. 
문서1 = np.array([0,1,1,1])
문서2 = np.array([1,0,1,1])
문서3 = np.array([2,0,2,2])

print('{:.2f}'.format(cos_sim(문서1, 문서2))) #문서1과 문서2의 코사인 유사도
print('{:.2f}'.format(cos_sim(문서1, 문서3))) #문서1과 문서3의 코사인 유사도
print('{:.2f}'.format(cos_sim(문서2, 문서3))) #문서2과 문서3의 코사인 유사도

# 코사인 유사도가 1이 나오면 코사인 유소의 값이 최대이다. 

0.67
0.67
1.00


In [ ]:
### 유사도를 이용한 추천시스템 구현하기 ###

In [73]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

data = pd.read_csv('/content/drive/MyDrive/머신러닝/movies.csv')
print(data['title'])
print(data['overview'])

0                                         Avatar
1       Pirates of the Caribbean: At World's End
2                                        Spectre
3                          The Dark Knight Rises
4                                    John Carter
                          ...                   
4798                                 El Mariachi
4799                                   Newlyweds
4800                   Signed, Sealed, Delivered
4801                            Shanghai Calling
4802                           My Date with Drew
Name: title, Length: 4803, dtype: object
0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
4798    El Mariachi just wants to play his guitar and ...
4799    A newl

In [76]:
data['overview'].isnull().sum()
data['title'].isnull().sum()

0

In [78]:
data['overview'] = data['overview'].fillna('')
data['overview'].isnull().sum()

0

In [ ]:
# TF-IDF
'''
[TF-IDF란?]
(TF란?)
: Term Frequency의 약자.
1개 문서 안에서 특정 단어의 등장 빈도를 의미합니다.
문장을 단어로 나누고,
전체 단어수에서 특정 단어가 얼마나 사용되었는지를 파악하여, 해당 문서의 성질을 파악하는 지표로 사용되죠.
단어라고 했는데, 주로 형태소 단위로 나누며,
또한 알파벳의 경우는 1글자 단어의 경우는 생략하는 경우가 많습니다.
 
(DF란?)
: Document Frequency의 약자입니다.
이는 특정 단어가 나타나는 문서 수를 나타냅니다.
TF와는 분류 스케일이 달라진 것이라 생각하세요. (단어가 주체인지 문서가 주체인지)
'''
# TF-IDF를 사용하는 이유
'''
(TF-IDF를 사용하는 이유)
: TF-IDF는 특징 추출 기법입니다.
머신 러닝시, 어떠한 데이터셋이 어떠한 성질을 지닌다로 나누는 것이 바로 특징인데,
이러한 특징(컴퓨터가 이해하기 쉬운 형식의 속성)을 추출하는 것이 바로 TF-IDF입니다.
 
특징 추출 기법은 여러가지가 있는데, 단순하게는, 단어의 빈도수를 기반으로 특징을 추출하는 CountVectorizer라는 기법이 있습니다.
하지만 이러한 count 기반의 특징 추출은, 조사, 관사처럼 많이 등장하는 단어들을 높게 쳐주기 때문에, 유의미한 결과를 얻기 어렵습니다.
 
TF-IDF는, 그 한계를 해결하고자 발전한 것으로,
단순히 많이 등장하는 빈도수로, 그 단어의 가치를 정하는 것이 아니라,
많이 등장하는 단어들에게는 일종의 패널티를 주어서, 단어 빈도의 스케일을 맞춰주는 기법이라 할수 있습니다.
(사람도 그렇죠? 무의미하게 말이 많은 사람의 정보는 어느정도 걸러듣게 되고, 필요할 때, 필요한 말을 딱 집어서 하는 사람의 말은 중요하게 듣게 되죠.)
 
즉, 단어 발생 빈도인 TF에서, 전체 발생 횟수에 따른 패널티를 부여해준 개념이 바로 TF-IDF라고 이해하세요
'''

In [87]:
# overview에 대해서 tf-idf 수행
# 벡터라이저가 단어들을 학습시킨다
tfidf = TfidfVectorizer(stop_words='english') 
# stop_words='english' 문서에 단어장을 생성할 때 무시할 수 있는 단어를 말한다. 
tfidf_matrix = tfidf.fit_transform(data['overview'])
tfidf.vocabulary_ # 벡터라이저가 학습된 단어를 출력한다. 
sorted(tfidf.vocabulary_.items()) # 단어사전을 정렬한다. 
print(tfidf_matrix.shape)
# 4803개의 영화를 표현하기 위해 총 21262개의 단어가 사용되어 있음
# 코사인 유사도를 사용하면서 문서의 유사도를 구해보자

(4803, 21262)


In [27]:
# sklearn.metrics.pairwise의 linear_kernel을 통해서 기존에 만들어 놓은 tfidf_matrix를 넣으면 코사인 유사도가 계산됩니다.
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(data.index, index=data['title']).drop_duplicates() 
# drop_dupllicates :  중복된 행들을 제거하고 유니크한 행들만 얻을 수 있다. 
print(indices.head())

title
Avatar                                      0
Pirates of the Caribbean: At World's End    1
Spectre                                     2
The Dark Knight Rises                       3
John Carter                                 4
dtype: int64


In [47]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    idx = indices[title]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬합니다. 
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아옵니다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return data['title'].iloc[movie_indices]

In [48]:
get_recommendations('Toy Story')

42                                   Toy Story 3
343                                  Toy Story 2
1779                      The 40 Year Old Virgin
891                              Man on the Moon
822                            Maid in Manhattan
1881                    The Shawshank Redemption
889                            The Thin Red Line
231                               Monsters, Inc.
197     Harry Potter and the Philosopher's Stone
1086                         Aliens in the Attic
Name: title, dtype: object